In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%shell
# Download SMDD

# Train set
# https://drive.google.com/file/d/1I6x_gWtu3WxOloK8k-tecXjV3XsRPYYO/view?usp=sharing_eip&ts=67b84547
# https://drive.usercontent.google.com/download?id=1I6x_gWtu3WxOloK8k-tecXjV3XsRPYYO&export=download&authuser=1&confirm=t&uuid=785a79b4-ffc3-4436-9ddb-068af853a6d4&at=APcmpowvgnzdC8Um8IU5X00GSm5D%3A1743709154940

rsync --progress /content/drive/MyDrive/mad/datasets/SMDD_release_train.zip /content/
mkdir -p /content/datasets/smdd-train/
unzip -q /content/SMDD_release_train.zip -d /content/datasets/smdd-train/


# Eval set
# https://drive.google.com/file/d/1V8libhAEMOL77gtUDtcruHIKBUy8ks6y/view?ts=67bf6247
# https://drive.usercontent.google.com/download?id=1V8libhAEMOL77gtUDtcruHIKBUy8ks6y&export=download&authuser=1&confirm=t&uuid=a2771e2e-7977-4b40-b032-1be0fa013f36&at=APcmpowppOG0vvYs8ve72fwnIelj%3A1743709146374

rsync --progress /content/drive/MyDrive/mad/datasets/SMDD_release_evaluation.zip /content/
mkdir -p /content/datasets/smdd-eval/
unzip -q /content/SMDD_release_evaluation.zip -d /content/datasets/smdd-eval/




# Download FRLL-Morphs dataset
wget -O /content/FRLL-Morphs.tar.gz https://zenodo.org/records/4415159/files/FRLL-Morphs.tar.gz?download=1
mkdir -p /content/datasets/frll-morphs/
tar -xvzf /content/FRLL-Morphs.tar.gz -C /content/datasets/frll-morphs/

# Add FRLL Bonafide part
wget -O /content/FRLL.zip https://figshare.com/ndownloader/articles/5047666/versions/5
unzip -q /content/FRLL.zip -d /content/datasets/frll/
mkdir -p /content/datasets/frll-morphs/facelab_london/raw/neutral/
mkdir -p /content/datasets/frll-morphs/facelab_london/raw/smiling/
unzip -q -j /content/datasets/frll/neutral_front.zip -d /content/datasets/frll-morphs/facelab_london/raw/neutral/
unzip -q -j /content/datasets/frll/smiling_front.zip -d /content/datasets/frll-morphs/facelab_london/raw/smiling/

find /content/datasets/frll-morphs/facelab_london/ -type f -not -name "*.jpg" -exec rm -f {} \;


In [ ]:
# Prepare the FRLL-Morphs csv files for each folder
%%shell

cd /content/datasets/frll-morphs/facelab_london
cp /content/drive/MyDrive/mad/datasets/scripts/frll_morphs_create_csv.py frll_morphs_create_csv.py
for dir in morph_*; do
    echo "Running for directory: $dir"
    python frll_morphs_create_csv.py $dir
done


In [ ]:
%%shell
python /content/drive/MyDrive/mad/code/SPL-MAD/train.py --train_csv '/content/datasets/smdd-train/train.csv' --prefix 'training-202503031020' --save_files '/content/drive/MyDrive/mad/code/SPL-MAD/'

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
GPU is available
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/drive/MyDrive/SPL-MAD/dataset.py:26: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  albumentations.Normalize(PRE__MEAN, PRE__STD, always_apply=True),
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this Data

In [ ]:
import os

train = "training-202503031020"
prefix = "testing-202503092037"

# Loop from 1 to 25
for i in range(1, 26):
    model_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/{i}.pth"
    output_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/{prefix}/{i}.csv"

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    log_file = f"smdd/normal/{prefix}"

    !python /content/drive/MyDrive/mad/code/SPL-MAD/test.py \
        --test_csv '/content/datasets/smdd-eval/evaluation.csv' \
        --model_path "{model_path}" \
        --output_path "{output_path}" \
        --prefix "{log_file}" \
        --save_files '/content/drive/MyDrive/mad/code/SPL-MAD/'

# Crop evaluation
for i in range(1, 26):
    model_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/{i}.pth"
    output_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/crop-{prefix}/{i}.csv"

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    log_file = f"smdd/crop/{prefix}"

    !python /content/drive/MyDrive/mad/code/SPL-MAD/test.py \
        --test_csv '/content/datasets/smdd-eval/evaluation_crop.csv' \
        --model_path "{model_path}" \
        --output_path "{output_path}" \
        --prefix "{log_file}" \
        --save_files '/content/drive/MyDrive/mad/code/SPL-MAD/'



GPU is available
/content/drive/MyDrive/SPL-MAD/dataset.py:54: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  albumentations.Normalize(PRE__MEAN, PRE__STD, always_apply=True),
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Number of test images: 40000
/content/drive/MyDrive/SPL-MAD/test.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://g

In [ ]:
# using FRLL-Morphs
import os

train = "training-202503031020"
prefix = "testing-202503131931_"

path_to_csv_files = "/content/datasets/frll-morphs/facelab_london/"
csv_files = ["morph_amsl", "morph_facemorpher", "morph_opencv", "morph_stylegan", "morph_webmorph"]

for csv_file in csv_files:

  # Loop from 1 to 25
  for i in range(1, 26):
      print(f'-----Current: {csv_file}:{i}')
      model_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/{i}.pth"
      output_path = f"/content/drive/MyDrive/mad/code/SPL-MAD/checkpoints/{train}/{prefix}/{csv_file}/{i}.csv"
      test_csv_path = f"{path_to_csv_files}/{csv_file}.csv"
      log_file = f"{csv_file}/{prefix}"

      # Ensure the output directory exists
      os.makedirs(os.path.dirname(output_path), exist_ok=True)

      !python /content/drive/MyDrive/mad/code/SPL-MAD/test.py \
          --test_csv "{test_csv_path}" \
          --model_path "{model_path}" \
          --output_path "{output_path}" \
          --prefix "{log_file}" \
          --save_files "/content/drive/MyDrive/mad/code/SPL-MAD/"



-----Current: morph_amsl:1
GPU is available
/content/drive/MyDrive/SPL-MAD/dataset.py:54: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  albumentations.Normalize(PRE__MEAN, PRE__STD, always_apply=True),
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Number of test images: 2379
/content/drive/MyDrive/SPL-MAD/test.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during